In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd





## Get Country Links

def getAllLinks(link):
    soup = getSoup(link)
    #Tracer()()
    tds = soup.findAll('td', {'class':'peopleTableCellName'})
    return ['http://ies.fsv.cuni.cz' + td.find('a')['href'] for td in tds]

links = getAllLinks('http://ies.fsv.cuni.cz/en/node/48')



## Get Top Places to Visit

In [15]:
def getSoup(link):
    r = requests.get(link)
    r.encoding = 'UTF-8'
    
    return BeautifulSoup(r.text,'lxml')
def getName(link):
    soup = getSoup(link)
    return soup.find('a').text

### Links


#### Country

'http://tripadvisor.com/Attractions-g274922-Activities-Slovakia.html'

In [129]:
soup = getSoup("https://www.tripadvisor.com/Search?geo=1&searchNearby=&pid=3826&redirect=&startTime=1555192604055&uiOrigin=MASTHEAD&q=Slovakia&supportedSearchTypes=find_near_stand_alone_query&enableNearPage=true&returnTo=__2F__&searchSessionId=6E04AA537D90EC250C122A440CB2639B1555192599769ssid&social_typeahead_2018_feature=true&sid=6E04AA537D90EC250C122A440CB2639B1555192607524")
div = soup.find('div', {'class':'location-meta-block'})
print(div)

None


In [ ]:
<div class="location-meta-block"><div class="result-title"><span>Slovakia</span></div><div class="review-count">256,357 reviews and opinions</div></div>

#### Things to do 

In [ ]:
<span class="ui_icon attractions brand-quick-links-QuickLinkTileItem__icon--2iguo"></span>
https://www.tripadvisor.com/

In [130]:
soup = getSoup("https://www.tripadvisor.com/")
span = soup.find('span', {'class':'ui_icon attractions brand-quick-links-QuickLinkTileItem__icon--2iguo'})
a = span.parent
ttdlink=a.get('href')
ta = 'http://tripadvisor.com' 
ta+ttdlink

'http://tripadvisor.com/Attractions'

In [122]:
soup = getSoup("https://www.tripadvisor.com/Home-g274922?fid=8de6eb68-5dc1-4991-96ac-1a5b90ed0b10")
span = soup.find('span', {'class':'ui_icon attractions brand-quick-links-QuickLinkTileItem__icon--2iguo'})
a = span.parent
ttdlink=a.get('href')
ta = 'http://tripadvisor.com' 
ta+ttdlink

'http://tripadvisor.com/Attractions-g274922-Activities-Slovakia.html'

### Titles


In [95]:
soup = getSoup("https://www.tripadvisor.com/Attractions-g274922-Activities-Slovakia.html")
divs = soup.findAll('div', {'class':'listing_title'}) 
text_chunks = {div.contents[1] for div in divs}
titles = {title.contents[0] for title in text_chunks}
print(titles)

{'Slavin', 'Church of St. Egidius', 'Lietava Castle', 'Modry Kostol', 'Slovak National Uprising Square', "St Elizabeth's Cathedral (Dom Svatej Alzbety)", 'Bratislava Old Town', 'Janosikove Diery', 'Rocky Mountain Lake', 'Belianska Cave', 'Town Tower', 'Museum of the Slovak National Uprising', 'Square of st. Egid', 'SNM - Bojnice Castle Museum', 'Kalvaria', 'Kosice Historical Centre', 'Waterfall Skok', 'Poprad Tourist information Center', 'Muzeum Zdiarsky Dom', 'Aquapark Tatralandia', 'Devin Castle', 'Jasna Nizke Tatry', 'Nitra Castle', 'Tatras National Park', 'The Clock Tower', 'Stary Smokovec–Hrebienok funicular', 'Demanovska Cave of Liberty', 'Zoo Kontakt', 'Trenciansky Hrad', 'Zelene pleso'}


## CLASS

In [ ]:
class TripAdvisor:
    '''
    Class containing methods for parsing IES websites from self.soup attribute.
    It is designed as a parent class for specific pages
    '''
    
    def __init__(self,link):
        self.link = link
        r = requests.get(link)
        r.encoding='UTF-8'
        self.soup = BeautifulSoup(r.text,'lxml')
    
    
    